In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# !rm -rf tf2.0-BERT/

In [12]:
# Download the bert code from github.
!git clone https://github.com/tensorflow/models.git
# Download necessary files from my github.
!git clone https://github.com/gnpdelta/tf2.0-BERT.git

Cloning into 'tf2.0-BERT'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 64 (delta 13), reused 64 (delta 13), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [4]:
# Install tf-nightly and bert depencency
# NOTE: Click "RESTART RUNTIME" button to restart the runtime environment after installing above.
!pip install tf-nightly-gpu
!pip install --user -r ./models/official/requirements.txt

     |████████████████████████████████| 517.2MB 29kB/s 
     |████████████████████████████████| 2.9MB 31.0MB/s 
     |████████████████████████████████| 460kB 42.9MB/s 
     |████████████████████████████████| 92kB 5.2MB/s 
     |████████████████████████████████| 174kB 11.1MB/s 
     |████████████████████████████████| 1.0MB 40.7MB/s 
     |████████████████████████████████| 21.6MB 26.4MB/s 
Obtaining pycocotools from git+https://github.com/cocodataset/cocoapi#egg=pycocotools&subdirectory=PythonAPI (from -r ./models/official/requirements.txt (line 25))
  Cloning https://github.com/cocodataset/cocoapi to ./src/pycocotools
  Running command git clone -q https://github.com/cocodataset/cocoapi /content/src/pycocotools
     |████████████████████████████████| 296kB 40.1MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-5.0.0-cp36-none-any.whl size=18684 sha256=b8a18236a16a62047d10539b1ef0a6e950f4810af15ec2ed76c6d219ce619c5f
  Stored in directory: /root/.cache/pip/wheels/01/7e/a9/b982d0fea

In [5]:
# Thers's some bug of tensorboard in colab so uninstall tensorboard and reinstall tb-nightly.
!pip uninstall tensorboard
!pip install --force-reinstall tb-nightly

Uninstalling tensorboard-2.2.0:
  Would remove:
    /usr/local/bin/tensorboard
    /usr/local/lib/python3.6/dist-packages/tensorboard-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorboard/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/tensorboard/uploader/flags_parser.py
Proceed (y/n)? y
  Successfully uninstalled tensorboard-2.2.0
  Using cached https://files.pythonhosted.org/packages/a6/63/a0b69e4c814581e52bfa76ec456fe7b910242b9497d8d57732ac1729da53/tb_nightly-2.3.0a20200406-py3-none-any.whl
     |████████████████████████████████| 112kB 3.4MB/s 
     |████████████████████████████████| 1.3MB 9.6MB/s 
     |████████████████████████████████| 307kB 20.3MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 92kB 7.4MB/s 
     |████████████████████████████████| 778kB 24.1MB/s 
     |████████████████████████████████| 2.9MB 33.8MB/s 
     |████████████████████████████████| 92kB 9.7MB/s

In [0]:
# 1. Add yourself data processing code in classifier_data_lib.py and create_finetuning_data.py  
# 2. Put them into "models" directory to replace original files.
!cp -ar tf2.0-BERT/code/data/*.py models/official/nlp/data/
!cp -ar tf2.0-BERT/code/bert/*.py models/official/nlp/bert/

In [7]:
# Set environment variables.
%set_env PYTHONPATH=./models
%set_env TASK_NAME=WEIBO
%set_env CONFIG_FILE=tf2.0-BERT/process_dir/models/hub_config
%set_env DATA_DIR=tf2.0-BERT/process_dir/dataset/train_eval_test_data
%set_env OUTPUT_DIR=tf2.0-BERT/output
%set_env MODEL_HUB_URL=https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/1

env: PYTHONPATH=./models
env: TASK_NAME=WEIBO
env: CONFIG_FILE=tf2.0-BERT/process_dir/models/hub_config
env: DATA_DIR=tf2.0-BERT/process_dir/dataset/train_eval_test_data
env: OUTPUT_DIR=tf2.0-BERT/output
env: MODEL_HUB_URL=https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/1


In [14]:
# Create train and eval data with tf_record format.
!python ./models/official/nlp/data/create_finetuning_data.py \
    --input_data_dir=${DATA_DIR} \
    --vocab_file=${CONFIG_FILE}/vocab.txt \
    --train_data_output_path=${DATA_DIR}/${TASK_NAME}_train.tf_record \
    --eval_data_output_path=${DATA_DIR}/${TASK_NAME}_eval.tf_record \
    --meta_data_file_path=${DATA_DIR}/${TASK_NAME}_meta_data \
    --fine_tuning_task_type=classification \
    --max_seq_length=180 \
    --classification_task_name=${TASK_NAME}

2020-04-08 09:36:23.137346: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
I0408 09:36:26.799381 139964357027712 classifier_data_lib.py:534] Writing example 0 of 89827
I0408 09:36:26.800556 139964357027712 classifier_data_lib.py:507] *** Example ***
I0408 09:36:26.800692 139964357027712 classifier_data_lib.py:508] guid: train-1
I0408 09:36:26.801644 139964357027712 classifier_data_lib.py:510] tokens: [CLS] # 上 海 肺 炎 # 近 期 上 海 也 出 现 了 一 例 新 型 冠 状 病 毒 感 染 的 肺 炎 疑 似 患 者 ， 小 伙 伴 们 出 门 要 记 得 带 好 口 罩 ， 少 去 人 员 密 集 的 场 所 噢 ！ 注 意 居 室 和 办 公 室 通 风 ， 多 多 锻 炼 身 体 ， 增 强 抵 抗 力 ， 健 健 康 康 的 哈 ～ ? 牙 齿 矫 正 李 彦 昂 医 生 发 布 于 疫 情 恐 慌 有 图 有 视 频 [SEP]
I0408 09:36:26.801824 139964357027712 classifier_data_lib.py:511] input_ids: 101 108 677 3862 5511 4142 108 6818 3309 677 3862 738 1139 4385 749 671 891 3173 1798 1094 4307 4567 3681 2697 3381 4638 5511 4142 4542 849 2642 5442 8024 2207 832 845 812 1139 7305 6206 6381 2533 2372 1962 1366 5388

In [0]:
# Start to train and eval...
!python ./models/official/nlp/bert/run_classifier.py \
    --mode='train_and_eval' \
    --input_meta_data_path=${DATA_DIR}/${TASK_NAME}_meta_data \
    --train_data_path=${DATA_DIR}/${TASK_NAME}_train.tf_record \
    --eval_data_path=${DATA_DIR}/${TASK_NAME}_eval.tf_record \
    --bert_config_file=${CONFIG_FILE}/bert_config.json \
    --train_batch_size=32 \
    --eval_batch_size=32 \
    --steps_per_loop=1 \
    --learning_rate=3e-5 \
    --num_train_epochs=2 \
    --model_dir=${OUTPUT_DIR} \
    --distribution_strategy=mirrored \
    --hub_module_url=${MODEL_HUB_URL} \
    --use_keras_compile_fit=True \
    --num_gpus=1

2020-04-08 09:43:28.746828: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/absl/flags/_validators.py:359: UserWarning: Flag --model_dir has a non-None default value; therefore, mark_flag_as_required will pass even if flag is not specified in the command line!
  'command line!' % flag_name)
2020-04-08 09:43:31.096704: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-08 09:43:31.121869: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-08 09:43:31.122693: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1544] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17

In [0]:
# generate predict dataset
!python ./models/official/nlp/data/do_pred_data.py \
    --input_data_dir=tf2.0-BERT/process_dir/dataset/predict_data \
    --predict_data_output_path=tf2.0-BERT/process_dir/dataset/predict_data/${TASK_NAME}_pred.tf_record \
    --vocab_file=${CONFIG_FILE}/vocab.txt \
    --meta_data_file_path=tf2.0-BERT/process_dir/dataset/predict_data/${TASK_NAME}_meta_data \
    --fine_tuning_task_type=classification \
    --max_seq_length=180 \
    --classification_task_name=${TASK_NAME}

In [0]:
# do predict!
!python ./models/official/nlp/bert/do_predict.py \
  --predict_data_path=tf2.0-BERT/process_dir/dataset/predict_data/${TASK_NAME}_pred.tf_record \
  --predict_output_dir=tf2.0-BERT/process_dir/dataset/predict_data/predict \
  --input_meta_data_path=tf2.0-BERT/process_dir/dataset/predict_data/${TASK_NAME}_meta_data \
  --predict_batch_size=32 \
  --max_seq_length=180 \


# 新段落

In [0]:
# view summaries in tensorboard
%reload_ext tensorboard
%tensorboard --logdir=${OUTPUT_DIR}/summaries/ 

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2020-04-08 04:33:22.994026: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/main.py", line 75, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 289, in main
    return runner(self.flags) or 0
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 305, in _run_serve_subcommand
    server = self._make_server()
  File "/usr/local/lib/python3